----
# Spectral clustering of GSCs
----
/cluster/projects/pughlab/projects/BTSCs_scRNAseq/Manuscript_G607removed/NatCan_Rebuttal/spectral_clustering

In [ ]:
setwd("~/Desktop/H4H/pughlab/projects/BTSCs_scRNAseq/Manuscript_G607removed/NatCan_Rebuttal/spectral_clustering/")

In [ ]:
###reformat DE markers
DE <- list.files(pattern="_DE_M")

markers <- list()

for (i in 1:length(DE)){
    
    print(DE[i])
    dat <- readRDS(DE[i])
    
    sample.markers <- list()
    
    for (j in 1:length(dat)){
    
    j <- as.numeric(j)
    print(j)
    dat[[j]]$Method <- as.character(names(dat)[j])
    dat[[j]] <- dat[[j]][dat[[j]]$p_val_adj < 0.01, ]
    aa <- data.frame((table(dat[[j]]$cluster)))
    colnames(aa) <- c("ClusterID", "NumMarkers")
    aa$Sample <- strsplit(names(dat[j]), split="[.]")[[1]][1]
    aa$Method <- strsplit(names(dat[j]), split="[.]")[[1]][2]
    sample.markers[[unique(aa$Method)]] <- aa
        
}
 
    markers[[unique(aa$Sample)]] <- do.call(rbind, sample.markers)
}

merge <- do.call(rbind, markers)
saveRDS(merge, file = "Combined_DE_markers.rds")

In [ ]:
###reformat sil widths
sil.files <- list.files(pattern="sil")

sils <- list()

for (i in 1:length(sil.files)){
    
    print(sil.files[i])
    dat <- readRDS(sil.files[i])
    
    sample.sils <- list()
    
    for (j in 1:length(dat)){
    
    aa <- data.frame(dat[[j]])
    colnames(aa) <- c("Avg.SilWidth")
    aa$Cluster <- names(dat[[j]])
    aa$Sample <- strsplit(names(dat[j]), split="[.]")[[1]][1]
    aa$Method <- strsplit(names(dat[j]), split="[.]")[[1]][2]
    sample.sils[[unique(aa$Method)]] <- aa
        
}
 
    sils[[unique(aa$Sample)]] <- do.call(rbind, sample.sils)
}

merge <- do.call(rbind, sils)
saveRDS(merge, file = "Combined_SilAvg.rds")

---
## 1.0 Pick optimal spectral clustering solution
---

Use silhouette width and marker genes

In [ ]:
DE <- readRDS("Combined_DE_markers.rds")
sil <- readRDS("Combined_SilAvg.rds")
samples <- unique(sil$Sample)
optimal.k <- c()
keep <- list()

for (i in 1:length(samples)){
    
    markers <- c()
    avg.sil <- c()
    k <- c()
    
    sample <- samples[i]
    print(sample)
    DE_sub <- DE[DE$Sample == sample, ]
    DE_sub <- DE_sub[grep("pectral", DE_sub$Method), ]
    
    ks <- unique(DE_sub$Method)
    for (j in 1:length(ks)){
        k[j] <- j+1
        markers[j] <- mean(DE_sub[DE_sub$Method==ks[j], ]$NumMarkers)
    }
    
    sil_sub <- sil[sil$Sample == sample, ]
    sil_sub <- sil_sub[grep("ectral", sil_sub$Method), ]
    methods <- unique(sil_sub$Method)
    for (x in 1:length(methods)){
        avg.sil[x] <- mean(sil_sub[sil_sub$Method==methods[x], ]$Avg.SilWidth)
    }
    
    b <- data.frame(k, markers, avg.sil)
    b <- b[b$markers > 20, ]
    keep[[sample]] <- b
    keep[[sample]]$sample <- sample
    optimal.k[i] <- b[b$avg.sil == max(b$avg.sil), ]$k
}


test <- do.call(rbind, keep)


---
## 2.0 Compare cluster number across methods
---

In [ ]:
### combine meta files across samples
meta.files <- list.files(pattern = "metadata")
meta <- list()
for (i in 1:length(meta.files)){
    
    meta[[meta.files[i]]] <- readRDS(meta.files[i])
    
}
meta <- do.call(rbind, meta)

In [ ]:
### combine sil width across samples
sil.files <- list.files(pattern = "silWidths")
sil <- list()
for (i in 1:length(sil.files)){
    
    sil[[sil.files[i]]] <- readRDS(sil.files[i])
    
}
#sil <- do.call(rbind, sil)

In [ ]:
test <- data.frame(sil[[1]][1])


In [ ]:
length(meta.files)